In [9]:
# import json
import os
import re
import lxml.etree as ET
from lxml import etree
from lxml import etree as ET
import dateparser
import datetime


def generate_id(tag):
    """Génère un identifiant unique pour une balise en fonction de son type."""
    if tag not in ids:
        ids[tag] = 1
    else:
        ids[tag] += 1
    return f"{tag}{ids[tag]}"

def write_comment(filename):
    filename = os.path.basename(filename)
    comment_text = f"New page added from {filename}"
    comment_element = ET.Comment(comment_text)
    body.append(comment_element)
    return comment_element



dossier_json = os.path.join(os.getcwd(), "json_data")
dossier_xml = os.path.join(os.getcwd(), "xml_data")

for fichier_json in os.listdir(dossier_json):
    if fichier_json.endswith(".json"):
        chemin_json = os.path.join(dossier_json, fichier_json)
        with open(chemin_json, "r", encoding="utf-8") as f:
            data = json.load(f)
            
            filename = os.path.basename(chemin_json)


            global div_sitting, div, body, ids
            ids = {"Partie_": 0, "n": 0, "s": 0, "note": 0}
            # Création de l'élément racine et le squelette du XML
            root = ET.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
            root.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr"


            xml_id = os.path.splitext(filename)[0]
            root.attrib["{http://www.w3.org/XML/1998/namespace}id"] = xml_id
            xml_id = os.path.splitext(filename)[0]
            root.attrib["{http://www.w3.org/XML/1998/namespace}id"] = xml_id

            teiHeader = ET.SubElement(root, "teiHeader")
            text_tei = ET.SubElement(root, "text")
            body = ET.SubElement(text_tei, "body")
            back = ET.SubElement(text_tei, "back")
            div_sitting = ET.SubElement(body, "div", attrib={"type": "sitting"})
            u_element = ET.Element("u")

            divs_cibles = []
            divs_cibles.append(div_sitting)
         
            
            
            
            for i in range(len(data)): 
                
                if "comment" in data[i]:  
                    
                    if data[i]['comment'] == 'part head' or data[i]['comment'] == "head part" :
                        
                        # Ajouter un élément 'part head' avec le contenu de la clé 'text_ocr'
                        div_part = ET.SubElement(div_sitting, "div", attrib={"type": "part", "corresp": "#pv"})
                        part_head = ET.SubElement(div_part, "head" )
                        part_head.text = data[i]['text_ocr'] 
                        div_part.addprevious(etree.Comment(generate_id("Partie_")))
                        divs_cibles.append(div_part)
                    
                    if  data[i]["comment"] == "u-end" or data[i]["comment"] == "u-end seg" or data[i]["comment"] ==  "seg u-end":
                        u_end = ET.Element("seg")
                        u_end.text = data[i]['text_ocr']
                        u_element.append(u_end)
                        
                        if u_element is not None:
                            u_element.tail = " "
                        u_element = ET.Element("u")
                        
                    if data[i]["comment"] == "u-beginning seg" :
                        u_beg = ET.Element("seg")
                        u_beg.text = data[i]['text_ocr']
                        u_element.append(u_beg)
                            
                        
                    if data[i]["comment"] == "u-beginning seg-beginning" :
                        u_beg_seg_beg = ET.Element("seg")
                        u_beg_seg_beg.text = data[i]['text_ocr']
                        u_element.append(u_beg_seg_beg)
                        
                        for div_cible in divs_cibles:
                            if div_cible is not None:
                                div_cible.append(u_element)

                    if data[i]["comment"] == "u-beginning seg-beginning incident" :
                        u_seg_beg_inc = ET.Element("seg")
                        u_seg_beg_inc.text = data[i]['text_ocr']
                        u_element.append(u_seg_beg_inc)
                        
                    if data[i]["comment"] == "u-end seg-end incident" or data[i]["comment"] == "seg-end incident" :
                                        
                        text_ocr = data[i]["text_ocr"]                        

                        # Rechercher l'indice de la première occurrence de "(" et de ")" dans le texte
                        start_index = text_ocr.find("(")
                        end_index = text_ocr.find(")")

                        if start_index != -1 and end_index != -1 and start_index < end_index:
                            # Extraire la partie du texte entre les guillemets
                            incident_text = text_ocr[start_index :end_index+1]

                            # Créer un élément <incident> et y ajouter le texte extrait
                            
                            incident_tag = ET.Element("incident")
                            incident_desc = ET.SubElement(incident_tag, "desc")
                            incident_desc.text = incident_text
                            incident_tag.tail = text_ocr[end_index + 1:]

                             # Insérer l'élément <incident> dans la bonne position en utilisant les méthodes d'insertion

                            if u_beg_seg_beg is not None:  
                                u_beg_seg_beg.text += "" + text_ocr[:start_index]
                                u_beg_seg_beg.insert(1, incident_tag)
                                incident_tag.tail = text_ocr[end_index + 1:]
                                u_element.append(u_beg_seg_beg)
                            else : 
                                u_element.text = text_ocr[:start_index]
                                u_element.append(incident_tag)
                                u_element.tail = text_ocr[end_index + 1:]
                                
                        if u_element is not None:
                            u_element.tail = " "
                        u_element = ET.Element("u")

                    if data[i]["comment"] == "u-beginning seg quote" : 
                        seg_cas = ET.Element("seg")
                        text_ocr = data[i]["text_ocr"]

                        # Rechercher l'indice de la première occurrence de "«" et de "»" dans le texte
                        start_index = text_ocr.find("«")
                        end_index = text_ocr.find("»")

                        if start_index != -1 and end_index != -1 and start_index < end_index:
                            # Extraire la partie du texte entre les guillemets
                            quote_text = text_ocr[start_index :end_index+1]

                            # Créer un élément <quote> et y ajouter le texte extrait
                            quote_seg = ET.SubElement(seg_cas, "quote")
                            quote_seg.text = quote_text

                             # Insérer l'élément <quote> dans la bonne position en utilisant les méthodes d'insertion
                            seg_cas.text = text_ocr[:start_index]
                            seg_cas.insert(1, quote_seg)
                            quote_seg.tail = text_ocr[end_index + 1:]
                            
                            u_element.append(seg_cas)
                        
                        else:
                            # Aucun guillemet trouvé ou l'ordre est incorrect, utiliser le texte tel quel
                            seg_cas.text = text_ocr                            
                        u_element.append(seg_cas)
                    
                    
                    if data[i]["comment"] == "quote-beginning seg" : 
                       
                        quote_beg = ET.SubElement(u_element, "quote")
                        quote_beg_seg = ET.SubElement(quote_beg,"seg")
                        quote_beg_seg.text = data[i]['text_ocr']
                        
                   
                    if data[i]["comment"] == "seg":
                        if i > 0 and "comment" in data[i-1] and data[i-1]["comment"] == "quote-beginning seg":
                            # Si la clé précédente est "quote-beginning seg", ajouter les balises seg à l'élément quote
                            while i < len(data) - 1 and "comment" in data[i+1] and data[i+1]["comment"] != "quote-end seg":
                                i += 1
                                if data[i]["comment"] == "seg":
                                    seg = ET.Element("seg")
                                    seg.text = data[i]['text_ocr']
                                    quote_beg.append(seg)
                        else:
                            # Sinon, ajouter les balises seg à l'élément u_element
                            seg = ET.Element("seg")
                            seg.text = data[i]['text_ocr']
                            u_element.append(seg)

                        
                    if data[i]["comment"] == "seg-beginning" :
                        seg_beg = ET.Element("seg")
                        seg_beg.text = data[i]['text_ocr']
                        u_element.append(seg_beg)


                    if data[i]["comment"] == "seg-end" :
                        if seg_beg is not None :
                            seg_beg.text += "" + data[i]['text_ocr']
                            u_element.append(seg_beg)       

                    if data[i]["comment"] == "u seg" : 
                        u_seg = ET.Element("seg")
                        u_seg.text = data[i]['text_ocr']
                        u_element.append(u_seg)

                        if u_element is not None:
                            u_element.tail = " "
                        u_element = ET.Element("u")

                    if data[i]["comment"] == "u seg quote" or data[i]["comment"] == "u-end seg quote":  
                        u_seg_quote = ET.Element("seg")
                                                
                        text_ocr = data[i]["text_ocr"]
                        # Rechercher l'indice de la première occurrence de "«" et de "»" dans le texte
                        start_index = text_ocr.find("«")
                        end_index = text_ocr.find("»")

                        if start_index != -1 and end_index != -1 and start_index < end_index:
                            # Extraire la partie du texte entre les guillemets
                            quote_text = text_ocr[start_index :end_index+1]

                            # Créer un élément <quote> et y ajouter le texte extrait
                            quote_seg = ET.SubElement(u_seg_quote, "quote")
                            quote_seg.text = quote_text

                             # Insérer l'élément <quote> dans la bonne position en utilisant les méthodes d'insertion
                            u_seg_quote.text = text_ocr[:start_index]
                            u_seg_quote.insert(1, quote_seg)
                            quote_seg.tail = text_ocr[end_index + 1:]
                            u_element.append(u_seg_quote)
                            
                        else:
                        # Aucun guillemet trouvé ou l'ordre est incorrect, utiliser le texte tel quel
                            u_seg_quote.text = text_ocr                            
                            u_element.append(u_seg_quote)

                        if u_element is not None:
                            u_element.tail = " "
                        u_element = ET.Element("u")


                    if data[i]["comment"] == "u seg incident" or data[i]["comment"] == "u-beginning seg incident" or data[i]["comment"] == "seg incident":                       
                        
                        text_ocr = data[i]["text_ocr"]                        

                        # Rechercher l'indice de la première occurrence de "(" et de ")" dans le texte
                        start_index = text_ocr.find("(")
                        end_index = text_ocr.find(")")

                        if start_index != -1 and end_index != -1 and start_index < end_index:
                            # Extraire la partie du texte entre les guillemets
                            incident_text = text_ocr[start_index :end_index+1]

                            # Créer un élément <incident> et y ajouter le texte extrait
                            u_seg_inc = ET.Element("seg")
                            incident_tag = ET.SubElement(u_seg_inc, "incident")
                            incident_desc = ET.SubElement(incident_tag, "desc")
                            incident_desc.text = incident_text

                             # Insérer l'élément <incident> dans la bonne position en utilisant les méthodes d'insertion
                            u_seg_inc.text = text_ocr[:start_index]
                            u_seg_inc.insert(1, incident_tag)
                            incident_tag.tail = text_ocr[end_index + 1:]
                            u_element.append(u_seg_inc)
                            
                        else : 
                            u_seg_inc = ET.Element("seg")
                            incident_tag = ET.SubElement(u_seg_inc, "incident")
                            incident_desc = ET.SubElement(incident_tag, "desc")
                            incident_desc.text = data[i]['text_ocr']
                            u_element.append(u_seg_inc)

                    if data[i]["comment"] == "u-end seg-end" or data[i]["comment"] == "seg-end u-end" :
                        
                        u_beg_seg_beg.text += "" + data[i]['text_ocr']
                        u_element.append(u_beg_seg_beg)
         
                        if u_element is not None:
                            u_element.tail = " "
                        u_element = ET.Element("u")

                  
                    if data[i]["comment"] == "u-end seg incident" or data[i]["comment"] == "seg incident u-end":
                        
                        text_ocr = data[i]["text_ocr"]                        

                        # Rechercher l'indice de la première occurrence de "(" et de ")" dans le texte
                        start_index = text_ocr.find("(")
                        end_index = text_ocr.find(")")

                        if start_index != -1 and end_index != -1 and start_index < end_index:
                            # Extraire la partie du texte entre les guillemets
                            incident_text = text_ocr[start_index :end_index+1]

                            # Créer un élément <incident> et y ajouter le texte extrait
                            u_end_seg_inc = ET.Element("seg")
                            incident_tag = ET.SubElement(u_end_seg_inc, "incident")
                            incident_desc = ET.SubElement(incident_tag, "desc")
                            incident_desc.text = incident_text

                             # Insérer l'élément <incident> dans la bonne position en utilisant les méthodes d'insertion
                            u_end_seg_inc.text = text_ocr[:start_index]
                            u_end_seg_inc.insert(1, incident_tag)
                            incident_tag.tail = text_ocr[end_index + 1:]
                            u_element.append(u_end_seg_inc)
                                
                        if u_element is not None:
                            u_element.tail = " "
                        u_element = ET.Element("u")
                        
                    
                    if data[i]["comment"] == "seg quote incident":
                        seg_quote = ET.Element("seg")
                        text_ocr = data[i]["text_ocr"]

                        # Rechercher l'indice de la première occurrence de "«" et de "»" dans le texte
                        start_index = text_ocr.find("«")
                        end_index = text_ocr.find("»")
                        
                        start_index1 = text_ocr.find("(")
                        end_index1 = text_ocr.find(")")

                        if start_index != -1 and end_index != -1 and start_index < end_index:
                            # Extraire la partie du texte entre les guillemets
                            quote_text = text_ocr[start_index:end_index+1]
                            incident_text = text_ocr[start_index1:end_index1+1]
                            # Créer un élément <quote> et y ajouter le texte extrait
                            quote_seg = ET.Element("quote")
                            quote_seg.text = quote_text
                            
                            quote_incident = ET.Element("incident")
                            quote_incident_desc = ET.SubElement(quote_incident, "desc")
                            quote_incident_desc.text = incident_text

                            # Insérer l'élément <quote> dans la bonne position en utilisant les méthodes d'insertion
                            seg_quote.text = text_ocr[:start_index]
                            seg_quote.append(quote_seg)
                            seg_quote.append(quote_incident)
                            quote_incident.tail = text_ocr[end_index1 + 1:]
                            
                        u_element.append(seg_quote)

                      
                    if data[i]["comment"] == "seg-end quote" :
                        seg_end_quote = ET.Element("seg")
                        seg_end_quote.text = data[i]['text_ocr']
                        
                        text_ocr = data[i]["text_ocr"]
                        # Rechercher l'indice de la première occurrence de "«" et de "»" dans le texte
                        start_index = text_ocr.find("«")
                        end_index = text_ocr.find("»")

                        if start_index != -1 and end_index != -1 and start_index < end_index:
                            # Extraire la partie du texte entre les guillemets
                            quote_text = text_ocr[start_index :end_index+1]

                            # Créer un élément <quote> et y ajouter le texte extrait
                            quote_end_seg = ET.SubElement(seg_end_quote, "quote")
                            quote_end_seg.text = quote_text

                             # Insérer l'élément <quote> dans la bonne position en utilisant les méthodes d'insertion
                            seg_end_quote.text = text_ocr[:start_index]
                            seg_end_quote.insert(1, quote_seg)
                            quote_end_seg.tail = text_ocr[end_index + 1:]
                        
                        if seg_beg is not None :
                            seg_beg.text += "" + seg_end_quote.text
                            u_element.append(seg_beg)    


                    if data[i]["comment"] == "quote-end seg" :
                        
                        quote_end_seg = ET.Element("seg")
                        quote_end_seg.text = data[i]['text_ocr']
                        quote_beg.append(quote_end_seg)
                        u_element.append(quote_beg)
                        
                        if quote_beg_seg is not None:
                            quote_end_seg.tail = " "
                        
                    
                    if data[i]['comment'] == 'comment seg' or data[i]["comment"] == "seg comment" :
                        # Ajouter un élément 'comment seg' avec le contenu de la clé 'text_ocr'
                        comment_note = ET.Element("note", attrib={"type": "comment"})
                        comment_seg = ET.SubElement(comment_note, "seg")
                        comment_seg.text = data[i]['text_ocr']
                        div_cible.append(comment_note)


                    if data[i]["comment"] == "opening seg" :
                        # Ajouter un élément 'opening seg' avec le contenu de la clé 'text_ocr'
                        opening_note = ET.Element("note", { "{http://www.w3.org/XML/1998/namespace}id": "CR_" + filename.split("_")[3] + "_" + generate_id("n")})
                        opening_seg = ET.SubElement(opening_note, "seg", { "{http://www.w3.org/XML/1998/namespace}id": generate_id("s")})
                        opening_seg.text = data[i]['text_ocr']  
                        div_cible.append(opening_note)

                    if data[i]['comment'] == 'table':
                        # Ajouter un élément 'table' avec le contenu de la clé 'text_ocr'
                        table = ET.Element("table")
                        row = ET.SubElement(table, "row")
                        cell = ET.SubElement(row, "cell")
                        cell.text = data[i]['text_ocr'] 
                        u_element.append(table)
                        
                    for div_cible in divs_cibles:
                        if div_cible is not None:
                            div_cible.append(u_element)
                        else : 
                            # Création de la division
                            div_cible_default = ET.Element("div")
                            div_cible_default.attrib["type"] = "default"
                            div_cible_default.append(u_element)

                            # Ajout de l'élément à div_sitting
                            body.append(div_cible_default)
                    
                    
                    

                    
                 # Création du fichier XML
            xml_tree = ET.ElementTree(root)
            output = xml_id + ".xml"
            xml_filename = os.path.join(dossier_xml, output)

            with open(xml_filename, "wb") as xml_file:
                    instruction1 = ' href="agoda_schema.rng" type="application/xml" schematypens="http://purl.oclc.org/dsdl/schematron"'
                    model_instr1 = ET.ProcessingInstruction("xml-model", instruction1)
                    root.addprevious(model_instr1)

                    instruction2 = ' href="agoda_schema.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"'
                    model_instr2 = ET.ProcessingInstruction("xml-model", instruction2)
                    root.addprevious(model_instr2)

                    xml_tree.write(xml_file, encoding="utf-8", xml_declaration=True, pretty_print=True)

# ___________________Fin du script en haut________Nettoyage, ci-dessous_______________________________________________________
            
# suppressions des tirets suivis de retours à la ligne

# Chemin d'accès au dossier contenant les fichiers XML
dossier_xml = os.path.join(os.getcwd(), "xml_data")

# Parcourir les fichiers XML dans le dossier
for nom_fichier in os.listdir(dossier_xml):
    if nom_fichier.endswith(".xml"):
        chemin_fichier = os.path.join(dossier_xml, nom_fichier)
        
        # Lire le contenu du fichier XML
        with open(chemin_fichier, "r") as fichier:
            contenu = fichier.read()
        
        # Supprimer les tirets suivis de retours à la ligne entre deux mots
        contenu_modifie = re.sub(r"(\w)-\n(\w)", r"\1\2", contenu)
        contenu_modifie = re.sub("-\n", "", contenu_modifie)
        contenu_modifie = re.sub(r"-\s", "", contenu_modifie)

        
        # Écrire le contenu modifié dans le fichier
        with open(chemin_fichier, "w") as fichier_modifie:
            fichier_modifie.write(contenu_modifie)
        
        #print(f"Le fichier '{nom_fichier}' a été modifié.")
